In [1]:
!pip install -q --upgrade pip
!pip install -U jaxlib==0.4.4+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 MB 6.8 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.6+cuda11.cudnn86
    Uninstalling jaxlib-0.4.6+cuda11.cudnn86:
      Successfully uninstalled jaxlib-0.4.6+cuda11.cudnn86
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 41.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
n

In [2]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.4/539.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.7/485.7 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.6/700.6 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=cc6faf76e98fea033d4982a00202b07062ff958d9a1db5c50ac4800f027c3882
  Sto

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [2]:
from network_config import get_2wl_network_configuration, get_gcn_network_configuration
from train import cross_validate, cross_entropy, save_GD_raw_results, save_core_results, prepare_data_subset, move_pattern_indices, train_loop
from data_preperation import prepare_tu_data_for_GCN, prepare_tu_data_for_2WL, load_gcn_data, load_twl_data

from jax import numpy as jnp
import matplotlib.pyplot as plt
from jax import jit, grad, vmap
import jax

import numpy as np
import os
import pandas as pd

In [3]:
# MUTAG 2 WL test hyper parameters
parameterization = "standard"

preprocessed_base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed"
repo_path = "/content/drive/MyDrive/Colab Notebooks/Masterarbeit/TWL_NTK"

utc_time = pd.Timestamp.utcnow()
dataset_name = "MUTAG"
layers, layer_wide, output_layer_wide, cv_folds, learning_rate, epochs = (5, 256, 1, 0, 0.001, 500)
data_path = preprocessed_base_path + f"/{dataset_name}"

# load preprocessed data
edge_features, pattern, Y, nb_edges, patterns_graph_map, graph_indx, pattern_graph_indx = load_twl_data(data_path, False)
Y = jnp.squeeze(Y)

# load the network configuration
init_fn, apply_fn, kernel_fn = get_2wl_network_configuration(layers, parameterization, layer_wide, output_layer_wide)

# loss function
loss = jit(cross_entropy)
grad_loss = jit(grad(lambda params, x, y, **kwargs: loss(y, (apply_fn(params, x, **kwargs))[:y.shape[0],:])))


pattern_segment_sum = jax.ops.segment_sum(jnp.ones(pattern_graph_indx.shape), pattern_graph_indx)

# pattern_moved = move_pattern_indices(
#     pattern[jnp.isin(pattern_graph_indx, graph_indx), :],
#     graph_indx,
#     pattern_segment_sum,
# )

nb_graphs = jnp.unique(graph_indx).shape[0]

res = train_loop(
    X_train = edge_features,
    graph_indx_train = graph_indx,
    Y_train = Y,
    pattern_train = pattern,
    pattern_graph_indx_train = pattern_graph_indx,
    nb_graphs_train = nb_graphs,
    X_val = edge_features,
    graph_indx_val = graph_indx,
    Y_val = Y,
    pattern_val = pattern,
    pattern_graph_indx_val = pattern_graph_indx,
    nb_graphs_val = nb_graphs,
    init_fn = init_fn,
    apply_fn = apply_fn,
    learning_rate = learning_rate,
    epochs = epochs,
    loss = loss,
    grad_loss = grad_loss,
    batch_size = 188,
    n_type = "twl",
)



	 train loss: 14456.70 | val loss: 14456.70 | train acc: 0.4053 | val acc: 0.4053
	 train loss: 936.50 | val loss: 936.50 | train acc: 0.6035 | val acc: 0.6035


KeyboardInterrupt: ignored

In [3]:
# The network configurations for the differen datsets
 
# dataset_name, layers, layer_wide, output_layer_wide, cv_folds, learning_rate, epochs, batch_size
GD_TWL_nn_config = {"MUTAG": (3, 32, 1, 10, 0.0005, 200, 188)}

GD_GCN_nn_config = {"MUTAG": (3, 32, 1, 10, 0.0005, 200, 188)}

# this is the same for all datasets and GD
parameterization = "standard"

In [5]:
# MUTAG 2 WL
preprocessed_base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed"
repo_path = "/content/drive/MyDrive/Colab Notebooks/Masterarbeit/TWL_NTK"

utc_time = pd.Timestamp.utcnow()
dataset_name = "MUTAG"
layers, layer_wide, output_layer_wide, cv_folds, learning_rate, epochs, batch_size = GD_TWL_nn_config[dataset_name]
data_path = preprocessed_base_path + f"/{dataset_name}"

# load preprocessed data
edge_features, pattern, Y, nb_edges, patterns_graph_map, graph_indx, pattern_graph_indx = load_twl_data(data_path, False)
Y = jnp.squeeze(Y)

# load the network configuration
init_fn, apply_fn, kernel_fn = get_2wl_network_configuration(layers, parameterization, layer_wide, output_layer_wide)

# loss function
loss = jit(cross_entropy)
grad_loss = jit(grad(lambda params, x, y, **kwargs: loss(y, (apply_fn(params, x, **kwargs))[:y.shape[0],:])))

train_results = cross_validate(
    "twl",
    edge_features,
    Y,
    graph_indx,
    pattern,
    pattern_graph_indx,
    cv_folds,
    init_fn,
    apply_fn,
    learning_rate,
    epochs,
    loss,
    grad_loss,
    batch_size
)

# save_GD_raw_results(
#     dataset_name, training_method, nn_type, epochs, utc_time.timestamp(), repo_path, cv_folds, train_results
# )
# save_core_results(np.array([np.array(i) for i in train_results[3]]), utc_time, dataset_name, training_method, nn_type, cv_folds, repo_path)

Start CV fold: 0
	 train loss: 95.74 | val loss: 89.15 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 16.71 | val loss: 9.56 | train acc: 0.5061 | val acc: 0.4543
	 train loss: 15.28 | val loss: 8.67 | train acc: 0.5044 | val acc: 0.4543
	 train loss: 10.23 | val loss: 6.87 | train acc: 0.3927 | val acc: 0.2410
	 train loss: 6.45 | val loss: 3.70 | train acc: 0.4956 | val acc: 0.4543
	 train loss: 3.93 | val loss: 2.47 | train acc: 0.4608 | val acc: 0.3934
	 train loss: 3.60 | val loss: 2.39 | train acc: 0.5445 | val acc: 0.5152
	 train loss: 3.03 | val loss: 1.91 | train acc: 0.5602 | val acc: 0.5152
	 train loss: 2.66 | val loss: 1.37 | train acc: 0.5863 | val acc: 0.6066
	 train loss: 2.37 | val loss: 1.88 | train acc: 0.5671 | val acc: 0.5457
	 train loss: 2.13 | val loss: 1.76 | train acc: 0.5671 | val acc: 0.5762
	 train loss: 1.92 | val loss: 1.68 | train acc: 0.5584 | val acc: 0.5762
	 train loss: 1.75 | val loss: 1.65 | train acc: 0.5532 | val acc: 0.6066
	 train loss: 1.

KeyboardInterrupt: ignored

In [4]:
# MUTAG GCN

preprocessed_base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed"
repo_path = "/content/drive/MyDrive/Colab Notebooks/Masterarbeit/GCN_NTK"

utc_time = pd.Timestamp.utcnow()
dataset_name = "MUTAG"
layers, layer_wide, output_layer_wide, cv_folds, learning_rate, epochs, batch_size = GD_GCN_nn_config[dataset_name]
data_path = preprocessed_base_path + f"/{dataset_name}"

# load preprocessed data
node_features, pattern, Y, _, _, graph_indx, pattern_graph_indx = load_gcn_data(data_path, False)
Y = jnp.squeeze(Y)

# load the network configuration
init_fn, apply_fn, kernel_fn = get_gcn_network_configuration(layers, parameterization, layer_wide, output_layer_wide)

# loss function
loss = jit(cross_entropy)
grad_loss = jit(grad(lambda params, x, y, **kwargs: loss(y, (apply_fn(params, x, **kwargs))[:y.shape[0],:])))

train_results = cross_validate(
    n_type = "gcn",
    X = node_features,
    Y = Y,
    graph_indx = graph_indx,
    pattern = pattern,
    pattern_graph_indx = pattern_graph_indx,
    cv_folds = cv_folds,
    init_fn = init_fn,
    apply_fn = apply_fn,
    learning_rate = learning_rate,
    epochs = epochs,
    loss = loss,
    grad_loss = grad_loss,
    batch_size = batch_size
)

# save_GD_raw_results(
#     dataset_name, training_method, nn_type, epochs, utc_time.timestamp(), repo_path, cv_folds, train_results
# )
# save_core_results(np.array([np.array(i) for i in train_results[3]]), utc_time, dataset_name, training_method, nn_type, cv_folds, repo_path)

/usr/local/lib/python3.9/dist-packages/neural_tangents/_src/stax/linear.py:439: UserWarning: Negative indices in `pattern` are considered as padding (i.e. ignored), unlike typical numpy negative indexing.
  warnings.warn('Negative indices in `pattern` are considered as padding '


Start CV fold: 0
	 train loss: 550.28 | val loss: 669.26 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 396.95 | val loss: 482.78 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 262.40 | val loss: 319.14 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 132.89 | val loss: 161.62 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 7.96 | val loss: 4.78 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 29.88 | val loss: 17.93 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 2.44 | val loss: 2.96 | train acc: 0.3509 | val acc: 0.2105
	 train loss: 2.63 | val loss: 1.58 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 1.50 | val loss: 0.91 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 0.66 | val loss: 0.62 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 0.65 | val loss: 0.60 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 0.65 | val loss: 0.54 | train acc: 0.6491 | val acc: 0.7895
	 train loss: 0.65 | val loss: 0.55 | train acc: 0.6491 | val acc: 0.7895
	 t